In [1]:
import panel as pn
import holoviews as hv
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '.')
import do_inference_single
pn.extension()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Create a pipe stream

In [3]:
pipe = hv.streams.Pipe(data=[])
tb = pn.pane.HTML("")

### Create some random buttons, text boxes

In [4]:
submit_button = pn.widgets.Button(name='Enter Query', button_type='primary')
submit_text = pn.widgets.TextInput(value=None)

def b_submit(event):
    pipe.send((submit_text.value))
    
submit_button.on_click(b_submit)

reset_button = pn.widgets.Button(name='Clear Query', button_type='primary')

def b_reset(event):
    submit_text.value = ""
    pipe.send((submit_text.value))
    
reset_button.on_click(b_reset)

### Make infrastructure for inference

In [5]:
dataset = hv.Dataset([])

In [6]:
inference = do_inference_single.DoInference()

Using configuration file : config.json


In [8]:
def select_data(ds, query): 
    df = ds.data
    if isinstance(query, list):
        select = pd.DataFrame([{'Info' : "Please Enter a query to continue"}])
    elif not str(query):
        select = pd.DataFrame([{'Info' : "Empty query"}])
    else :
        select = inference.do_inference(str(query), 10)
        
    bs = '''
        <head>

        <style>
        table {
          border: 2px solid black;
          border-collapse: collapse;
          width: 700px;
 
        }

        table td{
        border:2px solid black;
        width: 100px;
        word-wrap: break-word;
        }

        table th{
        border:2px solid black;
        font-weight: bold;
        width: 80px;
        font-size: 18px;
        text-align:center
        }
        
        .w {
         width: 350px;
        }
        
        </style>
        </head>
    '''  
    
    tb.object = bs + select.to_html().replace("dataframe","table").replace('<th>desc</th>','<th class="w">desc</th>')
    
    #print(try1.object)
    
    return hv.Div("")

filtered_ds = dataset.apply(select_data,  query=pipe.param.data)

### Layout data

In [12]:
p = pn.Column(
    pn.Column('# Do Inference'),
    pn.Column(
      submit_text, 
      pn.Row(submit_button, reset_button)
    ),
    pn.Column(
       filtered_ds.opts(width=1, height=1), tb
    )
)

pn.panel(p).servable(title='inf')

https://discourse.holoviz.org/t/how-do-i-use-custom-font-in-holoviews/303/10

In [10]:
#inference.do_inference("cycle")